In [1]:
! pip install requests beautifulsoup4
! pip install selenium
! pip install webdriver_manager
! pip install openpyxl

In [2]:
import os
import pandas as pd
from datetime import datetime
import openpyxl
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [3]:
print("Current Working Directory:", os.getcwd())

Current Working Directory: /Users/helgegeurtjacobusmoes/Desktop/Thesis Data


In [4]:
def load_data_with_urls(file_path):
    # Load the workbook
    wb = openpyxl.load_workbook(file_path, data_only=True)
    sheet = wb.active

    # List to store data along with URLs
    data = []
    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=2):  # Assuming headlines and publication info are in first two columns
        headline_cell, publication_cell = row
        headline = headline_cell.value
        publication = publication_cell.value
        url = headline_cell.hyperlink.target if headline_cell.hyperlink else None
        data.append({
            'Headline': headline,
            'Publication': publication,
            'URL': url
        })

    # Convert list to DataFrame
    return pd.DataFrame(data)

# Load the files into DataFrames with URLs
Trouw_df = load_data_with_urls("Trouw.xlsx")
Telegraaf_df = load_data_with_urls("De Telegraaf.xlsx")
AD_df = load_data_with_urls("Algemeen Dagblad.xlsx")
Volkskrant1_df = load_data_with_urls("De Volkskrant 1.xlsx")
NRC1_df = load_data_with_urls("NRC 1.xlsx")
Financieele1_df = load_data_with_urls("Financieele 1.xlsx")

# Sample 20 articles from each DataFrame
samples = [
    df.sample(20, random_state=1) for df in [Trouw_df, Telegraaf_df, AD_df, Volkskrant1_df, NRC1_df, Financieele1_df]
]

# Concatenate the sampled DataFrames
sampled_df = pd.concat(samples, ignore_index=True)

# Regex pattern to handle the extraction from 'Publication' column
regex_pattern = r'^(.*?), (.*?)(?:; Blz\. (?:NaN|\d+))?, (\d+) words$'
sampled_df[['News Outlet', 'Type of News', 'Word Count']] = sampled_df['Publication'].str.extract(regex_pattern)

# Clean up 'Type of News'
sampled_df['Type of News'] = sampled_df['Type of News'].str.replace(r'Blz\. \d+', '', regex=True)
sampled_df['Type of News'] = sampled_df['Type of News'].str.replace(r'[,0-9]+', '', regex=True)
sampled_df['Type of News'] = sampled_df['Type of News'].str.strip().str.rstrip(';')

# Remove duplicates
sampled_df = sampled_df.drop_duplicates()

# Display DataFrame
print(sampled_df.head())

                                            Headline   
0  Dit apparaat geeft je extra power tijdens het ...  \
1  Kunstmatige intelligentie verslaat artsen bij ...   
2                   Microsoft lijft OpenAI-topman in   
3  Waar komt het idee vandaan dat computers tot l...   
4                             Robot lacht om grapjes   

                                         Publication   
0              Trouw, Wetenschap; Blz. 17, 795 words  \
1                Trouw, Nederland; Blz. 8, 248 words   
2                   Trouw, Vandaag; Blz. 2, 94 words   
3  Trouw, Religie en Filosofie; Blz. 10, 11, 1379...   
4              Trouw, Buitenland; Blz. 12, 146 words   

                                                 URL News Outlet   
0  https://advance.lexis.com/api/document?collect...       Trouw  \
1  https://advance.lexis.com/api/document?collect...       Trouw   
2  https://advance.lexis.com/api/document?collect...       Trouw   
3  https://advance.lexis.com/api/document?collect...  

In [5]:
sampled_df

,Headline,Publication,URL,News Outlet,Type of News,Word Count
0,Dit apparaat geeft je extra power tijdens het ...,"Trouw, Wetenschap; Blz. 17, 795 words",https://advance.lexis.com/api/document?collect...,Trouw,Wetenschap,795
1,Kunstmatige intelligentie verslaat artsen bij ...,"Trouw, Nederland; Blz. 8, 248 words",https://advance.lexis.com/api/document?collect...,Trouw,Nederland,248
2,Microsoft lijft OpenAI-topman in,"Trouw, Vandaag; Blz. 2, 94 words",https://advance.lexis.com/api/document?collect...,Trouw,Vandaag,94
3,Waar komt het idee vandaan dat computers tot l...,"Trouw, Religie en Filosofie; Blz. 10, 11, 1379...",https://advance.lexis.com/api/document?collect...,Trouw,Religie en Filosofie,1379
4,Robot lacht om grapjes,"Trouw, Buitenland; Blz. 12, 146 words",https://advance.lexis.com/api/document?collect...,Trouw,Buitenland,146
...,...,...,...,...,...,...
115,Google zet kunstmatige intelligentie in bij ee...,"Het Financieele Dagblad, ONDERNEMEN; Blz. 13, ...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,ONDERNEMEN,264
116,Microsoft maakt radicale keuze voor AI,"Het Financieele Dagblad, PAGINA 10; Blz. 10, 1...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,PAGINA,1237
117,Hoe kunstmatige intelligentie het onderwijs ve...,"Het Financieele Dagblad, PAGINA 20; Blz. 20, 1...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,PAGINA,1359
118,Cyberwaakhond waarschuwt voor explosie phishin...,"Het Financieele Dagblad, PAGINA 1; Blz. 1, 741...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,PAGINA,741


In [6]:
# Write the final sample to an Excel file
sampled_df.to_excel("Randomized_Sampled_Articles.xlsx", index=False)

# Display the shape of the final DataFrame to verify the size
print("Final DataFrame shape:", sampled_df.shape)

Final DataFrame shape: (120, 6)


In [7]:
import pandas as pd

# Load the Excel file
file_path = '/Users/helgegeurtjacobusmoes/Desktop/Thesis Data/Randomized_Sampled_Articles.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows to understand its structure
print(data.head())

                                            Headline   
0  Dit apparaat geeft je extra power tijdens het ...  \
1  Kunstmatige intelligentie verslaat artsen bij ...   
2                   Microsoft lijft OpenAI-topman in   
3  Waar komt het idee vandaan dat computers tot l...   
4                             Robot lacht om grapjes   

                                         Publication   
0              Trouw, Wetenschap; Blz. 17, 795 words  \
1                Trouw, Nederland; Blz. 8, 248 words   
2                   Trouw, Vandaag; Blz. 2, 94 words   
3  Trouw, Religie en Filosofie; Blz. 10, 11, 1379...   
4              Trouw, Buitenland; Blz. 12, 146 words   

                                                 URL News Outlet   
0  https://advance.lexis.com/api/document?collect...       Trouw  \
1  https://advance.lexis.com/api/document?collect...       Trouw   
2  https://advance.lexis.com/api/document?collect...       Trouw   
3  https://advance.lexis.com/api/document?collect...  

## Retrieving Body and Publication Date

Geckodriver is used to retrieve the missing data that is needed to analyse the news articles. In this case, Mozilla Firefox is used, since it has the most favorable results and it retrieves the data the fastest. 

Retrieving Body and Published Date

Make sure that the driver is in the same work directory as the files that are being crawled. 

Retrieved from:
[https://www.selenium.dev/downloads/](https://www.selenium.dev/downloads/)

In [8]:
def find_geckodriver(start_path):
    # Define the name of the geckodriver file you're looking for
    geckodriver_filename = "geckodriver"  # Use "geckodriver.exe" on Windows

    # Walk through the directory
    for root, dirs, files in os.walk(start_path):
        if geckodriver_filename in files:
            return os.path.join(root, geckodriver_filename)
    return None  # Return None if not found

# Define the start path. For example, on macOS it might be something like below, adjust based on your username:
# start_path = '/Users/your_username/Desktop/thesis data'
# On Windows it might be something like below:
# start_path = 'C:\\Users\\your_username\\Desktop\\thesis data'

# Use os.path.expanduser to start at the current user's desktop if you want it more dynamic
start_path = os.path.expanduser('~/Desktop/thesis data')

# Find geckodriver
geckodriver_path = find_geckodriver(start_path)

if geckodriver_path:
    print("geckodriver found at:", geckodriver_path)
else:
    print("geckodriver not found.")


geckodriver found at: /Users/helgegeurtjacobusmoes/Desktop/thesis data/geckodriver


In [9]:
# Load the Excel file into a DataFrame
excel_path = '/Users/helgegeurtjacobusmoes/Desktop/thesis data/Randomized_Sampled_Articles.xlsx'
data = pd.read_excel(excel_path)

# Assuming URLs are in a column named 'URL', adjust if it's named differently
print(data['URL'].head())  # Display first few URLs to confirm correct loading

0    https://advance.lexis.com/api/document?collect...
1    https://advance.lexis.com/api/document?collect...
2    https://advance.lexis.com/api/document?collect...
3    https://advance.lexis.com/api/document?collect...
4    https://advance.lexis.com/api/document?collect...
Name: URL, dtype: object


def setup_driver():
    # Specify the path to GeckoDriver
    geckodriver_path = '/Users/helgegeurtjacobusmoes/Desktop/thesis data/geckodriver'
    service = Service(executable_path=geckodriver_path)
    driver = webdriver.Firefox(service=service)
    return driver

def fetch_article_details(driver, url):
    driver.get(url)
    body, publication_date = None, None
    try:
        # Wait for the publication date element to be present
        publication_date_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.SS_DocumentInfo:last-of-type"))
        )
        publication_date = publication_date_element.text if publication_date_element else "Publication date not found"

        # Get all paragraphs after the "Body" header within the article section
        body_elements = driver.find_elements(By.XPATH, "//h2[@id='JUMPTO_Body']/following-sibling::p")
        body_text_list = [element.text for element in body_elements if element.text.strip() != '']

        # Check if we have the ending phrase to stop at
        for paragraph in body_text_list:
            if "Bekijk de oorspronkelijke pagina" in paragraph:
                break
            if body:
                body += "\n" + paragraph
            else:
                body = paragraph

    except (NoSuchElementException, TimeoutException) as e:
        print(f"Error fetching details for URL {url}: {e}")

    return body, publication_date

def main():
    driver = setup_driver()

    # Load your data here
    data = pd.read_excel('/Users/helgegeurtjacobusmoes/Desktop/thesis data/Randomized_Sampled_Articles.xlsx')

    # Add new columns for body and publication date
    data['Body'] = None
    data['Publication Date'] = None

    # Process all entries in the dataframe
    for index, row in data.iterrows():
        if pd.notna(row['URL']):
            body, publication_date = fetch_article_details(driver, row['URL'])
            data.at[index, 'Body'] = body
            data.at[index, 'Publication Date'] = publication_date
            print(f"Processed index {index}: {row['URL']}")
        else:
            print(f"URL missing for index {index}")

    driver.quit()

    # Check the dataframe's head to confirm the columns are present
    print(data.head())
    
    # Save the updated dataframe to a new Excel file
    updated_file_path = '/Users/helgegeurtjacobusmoes/Desktop/thesis data/Updated_Randomized_Sampled_Articles.xlsx'
    data.to_excel(updated_file_path, index=False)

    print(f"Updated data has been saved to: {updated_file_path}")

if __name__ == "__main__":
    main()

## Cleaning data

The Body and the publications are cleaned in order to make them more readable.

In [ ]:
# Load the Excel file into a DataFrame
excel_path = '/Users/helgegeurtjacobusmoes/Desktop/thesis data/Updated_Randomized_Sampled_Articles.xlsx'


In [ ]:
# Load the Excel file into a DataFrame
excel_path = '/Users/helgegeurtjacobusmoes/Desktop/thesis data/Updated_Randomized_Sampled_Articles.xlsx'

# Load the Excel file into a DataFrame
updated_randomized_sampled_articles = pd.read_excel(excel_path)

updated_randomized_sampled_articles

In [ ]:
# Dictionary for Dutch to English month translation
dutch_months = {
    "januari": "January", "februari": "February", "maart": "March",
    "april": "April", "mei": "May", "juni": "June",
    "juli": "July", "augustus": "August", "september": "September",
    "oktober": "October", "november": "November", "december": "December"
}

# Function to translate Dutch month names to English and format the date
def translate_date(date_str):
    parts = date_str.split()
    if len(parts) >= 3:
        day, month_dutch, year = parts[:3]
        month_english = dutch_months.get(month_dutch.lower(), month_dutch)
        date_str_english = f"{day} {month_english} {year}"
        try:
            # Convert the English date string to a datetime object
            date_obj = datetime.datetime.strptime(date_str_english, "%d %B %Y")
            # Convert datetime object to the desired string format
            return date_obj.strftime("%d-%m-%Y")
        except ValueError:
            return date_str  # Return the original on failure
    return date_str  # Return the original if not enough parts

# Apply the translation and conversion function to the 'Publication Date' column
modified_data['Publication Date'] = modified_data['Publication Date'].apply(translate_date)

# Save the modified data back to Excel if needed
modified_data.to_excel("/Users/helgegeurtjacobusmoes/Desktop/thesis data/Updated_Randomized_Sampled_Articles.xlsx", index=False)

# Display the corrected data with the new date format
modified_data